# Agent

In [1]:
! pip install langchain_community langchain-openai langchainhub chromadb langchain tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
import json
import operator
from typing import Annotated, Sequence, TypedDict,Dict,Tuple
from langchain.prompts import PromptTemplate,ChatPromptTemplate,MessagesPlaceholder
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, Tool,AgentExecutor
from langchain_community.tools.tavily_search import TavilySearchResults

Setting the OpenAI and Tavily keys

In [3]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')

In [4]:
embeddings = OpenAIEmbeddings()

---------------------Buisness model canvas generator-----------------------

In [5]:
urls = [
    "https://www.healyconsultants.com/morocco-company-registration/setup-llc/",
    "https://life-in-morocco.com/registering-company-in-morocco"
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()

## making the chain tool

In [58]:
class Config():
    """
    Contains the configuration of the LLM.
    """
    model = 'gpt-4-turbo'
    llm = ChatOpenAI(temperature=0, model=model)

retrieval qa chain


In [59]:
cfg = Config()
qa = RetrievalQA.from_chain_type(
    llm=cfg.llm,
    chain_type="stuff",
    retriever = vectorstore.as_retriever()
)

Setting up the memroy for the agent

In [60]:
def setup_memory() -> Tuple[Dict, ConversationBufferMemory]:
    """
    Sets up memory for the  agent.
    :return a tuple with the agent keyword pairs and the conversation memory.
    """
    agent_kwargs = {
        "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
    }
    memory = ConversationBufferMemory(memory_key="memory", return_messages=True)
    return agent_kwargs, memory

making our agent

In [61]:
def setup_agent() -> AgentExecutor:
    """
    Sets up the tools for a function based chain.
    We have here the following tools:

    """
    cfg = Config()
    tools = [
        Tool(
            name="knowledge search",
            func=qa.run,
            description="useful for when you need information about questions about insurence. "
        ),
        Tool(
        name='web search',
        func=TavilySearchResults().run,
        description=(
            '''use this tool when you can't find the content in the knowledge base and you need more advenced search functionalities  '''
        ))

    ]
    agent_kwargs, memory = setup_memory()

    return initialize_agent(
        tools,
        cfg.llm,
        verbose=False,
        agent_kwargs=agent_kwargs,
        memory=memory
    )

This the agent where you perform all the interactions , it got chat memroy , output and other useful things

In [62]:
agent_executor = setup_agent()

In [76]:
agent_executor.memory.buffer

[HumanMessage(content="I'm building a startup about regulating banks using AI in casablanca"),
 AIMessage(content="**Business Model Canvas for AI Startup in Bank Regulation in Casablanca**\n\n1. **Customer Segments:**\n   - **Banks in Morocco:** Primary customers who need to comply with national and international regulatory standards.\n   - **Regulatory Bodies:** Such as Bank Al-Maghrib and the Ministry of Economy and Finance, which require tools to monitor and enforce compliance.\n\n2. **Value Propositions:**\n   - **Compliance Efficiency:** AI solutions to streamline and automate the compliance processes, reducing costs and time.\n   - **Risk Management:** Advanced analytics to identify and manage financial and operational risks.\n   - **Regulatory Insight:** AI-driven insights to help banks stay ahead of regulatory changes and requirements.\n\n3. **Channels:**\n   - **Direct Sales:** Engaging directly with banks and regulatory bodies through a dedicated sales team.\n   - **Online Pl

In [92]:
agent_executor.memory.clear()

In [93]:
# template="""You are "Moukawil.AI" an expert in assitant used by entrepreneur in the field of fintech.
# you can answer anyhting else and you answers should always be helpful and provide assitant for the users
# your job is to generate a Business Model Canvas for the user specific to the business

# these are the Details that you should customize for the user buisness:

# Customer Segments: Identify and describe the primary groups of people or organizations the business aims to serve.
# Value Propositions: Outline the unique value or benefits the business provides to its customer segments. What problems does it solve, or what needs does it fulfill?
# Channels: Specify the ways through which the business delivers its products or services to its customers. Include both direct and indirect channels.
# Customer Relationships: Describe the types of relationships the business establishes with its customers. How does it interact with them and foster loyalty?
# Revenue Streams: Detail the various ways the business generates revenue. Consider different pricing models, sales strategies, and sources of income.
# Key Resources: List the most important assets required to make the business model work, including physical, intellectual, human, and financial resources.
# Key Activities: Identify the crucial actions the business must undertake to deliver its value proposition, reach its market, and maintain its operations.
# Key Partnerships: Highlight the network of suppliers, partners, or collaborators that help the business succeed. What resources or activities do they provide?
# Cost Structure: Describe the major costs associated with operating the business. Consider fixed and variable costs, as well as the most significant expenditures.
# You have access to the following tools:
# \n\nknowledge search: useful for when you need information about questions about insurance.
# \nweb search: use this tool when you can't find the content in the knowledge base and you need more advanced search functionalities
# keep asking follow up questions to the user until you make the final answer
# make the answer specific to Morroco and you can include regulations, company names and other information that may be useful to the user
# \n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [knowledge search, web search]\n
# Action Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\n
# Thought: I now know the final answer\n
# Final Answer: the final answer to the original input question in json format\n\nBegin!\n\n
# Question: {input}\nThought:{agent_scratchpad}"""

template = """You are "Moukawil.AI," an expert assistant for fintech entrepreneurs. Your role is to generate a Business Model Canvas tailored to the user's business.

Business Model Canvas Customization Details:

- Customer Segments: Identify and describe the primary target groups.
- Value Propositions: Outline the unique value or benefits provided.
- Channels: Specify the delivery methods for products or services.
- Customer Relationships: Describe the types of relationships established.
- Revenue Streams: Detail the ways the business generates revenue.
- Key Resources: List crucial assets needed for the business model.
- Key Activities: Identify essential actions for delivering value and maintaining operations.
- Key Partnerships: Highlight important suppliers, partners, or collaborators.
- Cost Structure: Describe major costs, including fixed and variable expenditures.

You have access to the following tools:
- knowledge search: For insurance-related questions.
- web search: For advanced search needs beyond the knowledge base.

Process:

1. Keep asking follow-up questions until you have all the necessary details.
2. Customize the answer for Morocco, including relevant regulations and information.

Use the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [knowledge search, web search]\n
Action Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\n
Thought: I now know the final answer\n
Final Answer: the final answer to the original input question in json format\n\nBegin!\n\n

Question: {input}
Thought: {agent_scratchpad}"""


In [94]:
promptup = PromptTemplate(
    input_variables=['agent_scratchpad', 'input'],
    template = template
)

In [95]:
promptup.pretty_print()

You are "Moukawil.AI," an expert assistant for fintech entrepreneurs. Your role is to generate a Business Model Canvas tailored to the user's business.

Business Model Canvas Customization Details:

- Customer Segments: Identify and describe the primary target groups.
- Value Propositions: Outline the unique value or benefits provided.
- Channels: Specify the delivery methods for products or services.
- Customer Relationships: Describe the types of relationships established.
- Revenue Streams: Detail the ways the business generates revenue.
- Key Resources: List crucial assets needed for the business model.
- Key Activities: Identify essential actions for delivering value and maintaining operations.
- Key Partnerships: Highlight important suppliers, partners, or collaborators.
- Cost Structure: Describe major costs, including fixed and variable expenditures.

You have access to the following tools:
- knowledge search: For insurance-related questions.
- web search: For advanced search n

In [96]:
agent_executor.agent.llm_chain.prompt = promptup

In [97]:
a = agent_executor.invoke("""I'm building a startup about regulating banks using AI in casablanca, give me customized BMC""")

In [98]:
a['output']

'{\n  "Customer Segments": "Banks in Casablanca and potentially other parts of Morocco, focusing on those needing to enhance their regulatory compliance and risk management.",\n  "Value Propositions": "Advanced AI-driven tools for real-time compliance monitoring, risk assessment, and fraud detection, which can significantly reduce the risk of penalties and improve operational efficiencies.",\n  "Channels": "Direct sales to banks, online platforms for demonstrations and software downloads, and participation in fintech and banking conferences.",\n  "Customer Relationships": "Establishing long-term relationships through continuous support and updates, personalized training sessions for bank staff, and dedicated account managers.",\n  "Revenue Streams": "Subscription-based models for software access, fees for consulting and customization services, and possibly licensing fees for proprietary algorithms.",\n  "Key Resources": "AI technology expertise, software development tools, compliance a

using TruLens 🦑 to evaluate the performence of the QA (we used only the qa chain for the evaluation and we will include more advanced evaluationg in the future )

In [ ]:
!pip install trulens_eval

In [ ]:
from trulens_eval.feedback.provider import OpenAI
import numpy as np
from trulens_eval import TruChain, Feedback, Tru

tru = Tru()
tru.reset_database()


openai = OpenAI()

from trulens_eval.app import App
context = App.select_context(qa)

from trulens_eval.feedback import Groundedness
grounded = Groundedness(groundedness_provider=OpenAI())
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons)
    .on(context.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

f_qa_relevance = Feedback(openai.relevance).on_input_output()
f_context_relevance = (
    Feedback(openai.qs_relevance)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.
✅ In groundedness_measure_with_cot_reasons, input source will be set to __record__.app.retriever.get_relevant_documents.rets.collect() .
✅ In groundedness_measure_with_cot_reasons, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In qs_relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In qs_relevance, input statement will be set to __record__.app.retriever.get_relevant_documents.rets .


In [ ]:
tru_recorder = TruChain(qa,
    app_id='ChatApp',
    feedbacks=[f_qa_relevance, f_context_relevance, f_groundedness])

In [ ]:
with tru_recorder as recording:
    llm_response = qa.run(caption + 'I had this terrible accident, tell me what I can do?')
display(llm_response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


"Based on the information provided, here are some steps you can take:\n\n1. Contact your insurance company: Report the accident to your insurance company as soon as possible. They will guide you through the process of filing a claim. \n\n2. Collision coverage: If you have collision coverage, it should cover the damage to your vehicle, subject to your deductible. This coverage is designed to provide payments to repair the damaged vehicle, or payment of the cash value of the vehicle if it is not repairable or totaled.\n\n3. Comprehensive coverage: If the accident was caused by factors other than collision (like weather, vandalism, etc.), and you have comprehensive coverage, it should cover the damage to your vehicle, subject to your deductible.\n\n4. Legal advice: Depending on the circumstances of the accident, you may want to consult with a lawyer, especially if there were injuries involved or if you're unsure about your insurance coverage.\n\n5. Document everything: Take pictures of th

In [ ]:
rec = recording.get()

In [ ]:
for feedback, feedback_result in rec.wait_for_feedback_results().items():
    print(feedback.name, feedback_result.result)

relevance 0.8
qs_relevance 0.2
groundedness_measure_with_cot_reasons 0.2857142857142857


In [ ]:
tru.get_leaderboard(app_ids=["ChatApp"])

,relevance,groundedness_measure_with_cot_reasons,qs_relevance,latency,total_cost
app_id,,,,,
ChatApp,0.8,0.285714,0.2,12.0,0.0486


In [ ]:
a = agent_executor.invoke('some guy just hit my car from the back in the red light , it is severly damaged what I should do? ')

In [ ]:
tru.run_dashboard()

Starting dashboard ...
npx: installed 22 in 3.984s

Go to this url and submit the ip given here. your url is: https://cute-times-try.loca.lt

  Submit this IP Address: 34.106.132.124



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

In [ ]:
tru.stop_dashboard()

Dashboard closed.
Dashboard closed.
